<a href="https://colab.research.google.com/github/samuelaojih/Google-Colab/blob/main/FNF_Congo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
pip install earthengine-api


In [ ]:
ee.Authenticate()
ee.Initialize(
    project ='ee-samuelojihcool',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

In [ ]:


# 1. Load your Congo Basin boundary (update to your asset path)
congo_basin = ee.FeatureCollection('projects/ee-samuelojihcool/assets/Congo_Basin_Shapefile')  # Replace with actual asset

# 2. Load the JAXA ALOS PALSAR FNF 2015 dataset
jaxa_fnf_2015 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF') \
    .filterDate('2015-01-01', '2015-12-31') \
    .first()

# 3. Clip to the Congo Basin
fnf2015_clipped = jaxa_fnf_2015.clip(congo_basin)

# 4. Create forest and non-forest masks
forest = fnf2015_clipped.eq(1).selfMask()
non_forest = fnf2015_clipped.eq(2).selfMask()

# 5. Calculate area (in square kilometers)
pixel_area = ee.Image.pixelArea()

forest_area = pixel_area.updateMask(forest).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=congo_basin.geometry(),
    scale=30,
    maxPixels=1e13
).get('area')

non_forest_area = pixel_area.updateMask(non_forest).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=congo_basin.geometry(),
    scale=30,
    maxPixels=1e13
).get('area')

# Convert square meters to square kilometers
forest_area_km2 = ee.Number(forest_area).divide(1e6).getInfo()
non_forest_area_km2 = ee.Number(non_forest_area).divide(1e6).getInfo()

# 6. Print the results
print(f"Forest Area in 2015 (sq.km): {forest_area_km2:.2f}")
print(f"Non-Forest Area in 2015 (sq.km): {non_forest_area_km2:.2f}")

# 7. Optional: Export the image to Google Drive
export_task = ee.batch.Export.image.toDrive(
    image=fnf2015_clipped,
    description='CongoBasin_Forest_NonForest_2015',
    folder='Congo_Basin',
    region=congo_basin.geometry(),
    scale=30,
    maxPixels=1e13
)

# Start the export
export_task.start()
print("Export started. Check your Google Drive and Earth Engine tasks tab.")


Forest Area in 2015 (sq.km): 2704124.36
Non-Forest Area in 2015 (sq.km): 911815.23
Export started. Check your Google Drive and Earth Engine tasks tab.


In [ ]:
# Load Congo Basin boundary (replace with your actual asset ID)
congo_basin = ee.FeatureCollection('projects/ee-samuelojihcool/assets/Congo_Basin_Shapefile')  # Replace

# Load JAXA Forest/Non-Forest images for 2010 and 2015
fnf_2010 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF') \
    .filterDate('2010-01-01', '2010-12-31') \
    .first() \
    .clip(congo_basin)

fnf_2015 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF') \
    .filterDate('2015-01-01', '2015-12-31') \
    .first() \
    .clip(congo_basin)

# Create forest masks
forest_2010 = fnf_2010.eq(1)
forest_2015 = fnf_2015.eq(1)

# Create forest change map
# -1 = Loss (forest in 2010 but not in 2015)
#  0 = No Change
#  1 = Gain (not forest in 2010 but forest in 2015)
change_map = forest_2015.subtract(forest_2010)

# Set visualization parameters
change_viz = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'lightgray', 'green'],  # Loss, No change, Gain
}

# Display using geemap
Map = geemap.Map(center=[0, 20], zoom=5)
Map.addLayer(change_map, change_viz, 'Forest Change 2010-2015')
Map.addLayer(congo_basin, {}, 'Congo Basin Boundary')
Map.add_legend(
    title="Forest Change (2010–2015)",
    labels=["Loss", "No Change", "Gain"],
    colors=['#FF0000', '#D3D3D3', '#008000']  # Using hex color codes
)
Map

Map(center=[0, 20], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…